In [1]:
import os
import mat_properties as prop
import numpy as n
import pandas as pd
import cotel
import nasos
import time
from scipy.optimize import root

#Время начала расчёта
start_time = time.time()

#Основные константы
calctolerance=10**-3
calcmethod = "hybr"
KPDnasos = 0.9
KPD=0.99

#таблица ном режима
gas_streams0   = pd.read_excel('streams0.xlsx', sheet_name='gas',  index_col=0)
water_streams0 = pd.read_excel('streams0.xlsx', sheet_name='water',index_col=0)
#таблица задаваемых значений (заменится на интерфейс)
gas_streamsZAD    = pd.read_excel('streams.xlsx',  sheet_name='gas',  index_col=0)
water_streamsZAD  = pd.read_excel('streams.xlsx',  sheet_name='water',index_col=0)
#рабочая таблица (=номинал в 1 итерации)
gas_streams = gas_streams0.copy()
water_streams = water_streams0.copy()


#чтение с вводного экселя входных данных
Tgas = gas_streamsZAD.loc['GTU-PEVD','T']
Pgas = gas_streamsZAD.loc['GTU-PEVD','P']
Ggas = gas_streamsZAD.loc['GTU-PEVD','G']
Propgas = gas_streamsZAD.loc['GTU-PEVD','N2':'Ar']
Pvd = water_streamsZAD.loc['IVD-PEVD','P']
Pnd = water_streamsZAD.loc['IND-PPND','P']
Twat0 = water_streamsZAD.loc['X-GPK','T']

#запись состава газов в раб табл
gas_streams['N2'] = Propgas[0]
gas_streams['O2'] = Propgas[1]
gas_streams['CO2'] = Propgas[2]
gas_streams['H2O'] = Propgas[3]
gas_streams['Ar'] = Propgas[4]


#Состав газов в номинальном режиме и в частичной нагрузке
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
fractiongas0 = (gas_streams0.at['GTU-PEVD','N2'],gas_streams0.at['GTU-PEVD','O2'],gas_streams0.at['GTU-PEVD','CO2'],gas_streams0.at['GTU-PEVD','H2O'],gas_streams0.at['GTU-PEVD','Ar'])
fractiongas  = (gas_streams.at['GTU-PEVD','N2'],gas_streams.at['GTU-PEVD','O2'],gas_streams.at['GTU-PEVD','CO2'],gas_streams.at['GTU-PEVD','H2O'],gas_streams.at['GTU-PEVD','Ar'])

#Считывание рефпропа
RP = prop.init_REFPROP(r'C:\Program Files (x86)\REFPROP')
os.environ['RPPREFIX'] = r'C:/Program Files (x86)/REFPROP'
gas1 = prop.Materials_prop(gasmix,fractiongas, prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)
gas0 = prop.Materials_prop(gasmix,fractiongas0, prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)
water=prop.Materials_prop('water',[1.0,0,0,0,0], prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)

#Задание в раб табл параметров газа (1 строка)
gas_streams.loc['GTU-PEVD','T'] = Tgas
gas_streams['G'] = Ggas
gas_streams['P'] = Pgas
gas_streams.loc['GTU-PEVD','H'] = gas1.p_t(Pgas,Tgas)['h']

#Задание в раб табл параметров по воде
water_streams.loc['IVD-PEVD':'PEN-EVD','P'] = Pvd
water_streams.loc['IND-PPND':'X-GPK','P'] = Pnd
water_streams.loc['BND-PEN','P'] = Pnd
water_streams.loc['X-GPK','T'] = Twat0

#Расчёт параметров на насыщении в испарительных поверхностях (недогрев до нас на входе 3°С)
water_streams.loc['IVD-PEVD','T'] = water.p_q(Pvd,1)['T']
water_streams.loc['IVD-PEVD','H'] = water.p_q(Pvd,1)['h']
water_streams.loc['IND-PPND','T'] = water.p_q(Pnd,1)['T']
water_streams.loc['IND-PPND','H'] = water.p_q(Pnd,1)['h']
water_streams.loc['EVD-IVD','T'] = water.p_q(Pvd,1)['T']-3
water_streams.loc['GPK-IND','T'] = water.p_q(Pnd,1)['T']-3
water_streams.loc['EVD-IVD','H'] = water.p_t(Pvd,water_streams.loc['EVD-IVD','T'])['h']
water_streams.loc['GPK-IND','H'] = water.p_t(Pnd,water_streams.loc['GPK-IND','T'])['h']


it=30
for k in range(it):
    ######################################################################
    #Связвка высокого давления
    for j in range(it):
        for i in range(it):
            PEVD = cotel.heatex('GTU-PEVD','PEVD-IVD','IVD-PEVD','PEVD-DROS',       KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
            gas_streams.loc['PEVD-IVD','T':'G'] = [PEVD[0],PEVD[1],PEVD[2],PEVD[3]]
            water_streams.loc['PEVD-DROS','T':'G'] = [PEVD[4],PEVD[5],PEVD[6],PEVD[7]]
            IVD  = cotel.vapor ('PEVD-IVD','IVD-EVD','EVD-IVD','IVD-PEVD',           KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
            gas_streams.loc['IVD-EVD','T':'G'] = [IVD[0],IVD[1],IVD[2],IVD[3]]
            water_streams.loc['IVD-PEVD','T':'G'] = [IVD[4],IVD[5],IVD[6],IVD[7]]
            water_streams.loc['PEVD-DROS':'PEN-EVD','G'] = IVD[7]
            Qgas = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['PEVD-IVD','H'])
            Qwat = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['IVD-PEVD','H'])
            print('dQ/Q ПЕВД+ИВД',((Qgas-Qwat)/Qgas))
            if  abs((Qgas-Qwat)/Qgas) < calctolerance:
                break;
        EVD = cotel.heatex('IVD-EVD','EVD-PPND','PEN-EVD','EVD-IVD',               KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
        gas_streams.loc['EVD-PPND','T':'G'] = [EVD[0],EVD[1],EVD[2],EVD[3]]
        water_streams.loc['EVD-IVD','T':'G'] = [EVD[4],EVD[5],EVD[6],EVD[7]]

        Qgas1 = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['IVD-EVD','H'])
        Qwat1 = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['EVD-IVD','H'])
        print('dQ/Q ПЕВД+ИВД+ЭВД',((Qgas1-Qwat1)/Qgas1))
        if  abs((Qgas1-Qwat1)/Qgas1) < calctolerance:
            break;
    print('Степень сухости пара в ЭВД: ',water.p_h(water_streams.at['EVD-IVD','P'],water_streams.at['EVD-IVD','H'])['Q'])
    print("ВД:--- %s сек. ---" % round((time.time() - start_time)))
    ######################################################################


    ######################################################################
    #Связка низкого давления
    for j in range(it):
        #Связка ППНД+ИНД
        for i in range(it):
            #Расчёт ППНД
            PPND = cotel.heatex('EVD-PPND','PPND-IND','IND-PPND','PPND-TURB',    KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
            gas_streams.loc['PPND-IND','T':'G']    = [PPND[0],PPND[1],PPND[2],PPND[3]]
            water_streams.loc['PPND-TURB','T':'G'] = [PPND[4],PPND[5],PPND[6],PPND[7]]
            #Расчёт ИНД
            IND  = cotel.vaporND('PPND-IND','IND-GPK','GPK-IND','IND-PPND',      KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
            gas_streams.loc['IND-GPK','T':'G']    = [IND[0],IND[1],IND[2],IND[3]]
            water_streams.loc['IND-PPND','T':'G'] = [IND[4],IND[5],IND[6],IND[7]]
            #Переопределение расхода пара
            water_streams.loc['PPND-TURB':'IND-PPND','G'] = IND[7]
            Qgas = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['IND-GPK','H'])
            Qwat = water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
            print('dQ/Q ППНД+ИНД',((Qgas-Qwat)/Qgas))
            if  abs((Qgas-Qwat)/Qgas) < calctolerance:
                break;
        #ПЭН
        water_streams.loc['BND-PEN','T':'G'] = [IND[9],IND[10],IND[11],IND[12]]
        PEN=nasos.nasos('BND-PEN','PEN-EVD',water,KPDnasos,water_streams).calc()
        water_streams.loc['PEN-EVD','T':'G'] = [PEN[0],PEN[1],PEN[2],PEN[3]]

        #Расчёт расхода в ГПК
        water_streams.loc['X-GPK','G'] = water_streams.at['PPND-TURB','G']+water_streams.at['PEVD-DROS','G']
        water_streams.loc['GPK-IND','G'] = water_streams.at['X-GPK','G']
        #Расчёт ГПК
        GPK = cotel.heatex('IND-GPK','GPK-out','X-GPK','GPK-IND',           KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
        gas_streams.loc['GPK-out','T':'G']   = [GPK[0],GPK[1],GPK[2],GPK[3]]
        water_streams.loc['GPK-IND','T':'G'] = [GPK[4],GPK[5],GPK[6],GPK[7]]

        Qgas1 = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['GPK-out','H'])
        Qwat1 = water_streams.at['GPK-IND','G']*(water_streams.at['GPK-IND','H']-water_streams.at['X-GPK','H'])+water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
        print('dQ/Q ППНД+ИНД+ГПК',abs((Qgas1-Qwat1)/Qgas1))
        if  abs((Qgas1-Qwat1)/Qgas1) < calctolerance:
            break;
    print('Степень сухости пара в ГПК: ',water.p_h(water_streams.at['GPK-IND','P'],water_streams.at['GPK-IND','H'])['Q'])
    print("НД:--- %s сек. ---" % round((time.time() - start_time)))
    ######################################################################
    
    
    
    Qgasall = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['GPK-out','H'])
    Qwatall = water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['X-GPK','H'])+water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['X-GPK','H'])-water_streams.at['BND-PEN','G']*(water_streams.at['PEN-EVD','H']-water_streams.at['BND-PEN','H'])
    print('dQ/Qsumm',abs((Qgasall-Qwatall)/Qgasall))
    if  abs((Qgasall-Qwatall)/Qgasall)*100 < calctolerance:
        break;


print("fin:--- %s сек. ---" % round((time.time() - start_time)))

dQ/Q ПЕВД+ИВД 0.07971395707355534
dQ/Q ПЕВД+ИВД -0.02678812655234902
dQ/Q ПЕВД+ИВД 0.008279293453519108
dQ/Q ПЕВД+ИВД -0.002628579549856942
dQ/Q ПЕВД+ИВД 0.0008275344167286308
dQ/Q ПЕВД+ИВД+ЭВД 0.01475524755478742
dQ/Q ПЕВД+ИВД -0.0057579899365249225
dQ/Q ПЕВД+ИВД 0.0018398393031001483
dQ/Q ПЕВД+ИВД -0.0005913382605926116
dQ/Q ПЕВД+ИВД+ЭВД 0.019256492807292833
dQ/Q ПЕВД+ИВД -0.028922131866683624
dQ/Q ПЕВД+ИВД 0.009164880521329829
dQ/Q ПЕВД+ИВД -0.002989627516345762
dQ/Q ПЕВД+ИВД 0.0009661704901522975
dQ/Q ПЕВД+ИВД+ЭВД -0.007430491941344046
dQ/Q ПЕВД+ИВД 0.01091117270185628
dQ/Q ПЕВД+ИВД -0.003551597669920216
dQ/Q ПЕВД+ИВД 0.0011432297184121628
dQ/Q ПЕВД+ИВД -0.00036932614594861496
dQ/Q ПЕВД+ИВД+ЭВД 0.0028341937849260543
dQ/Q ПЕВД+ИВД -0.004212771524726669
dQ/Q ПЕВД+ИВД 0.001357228774583987
dQ/Q ПЕВД+ИВД -0.0004391312610735033
dQ/Q ПЕВД+ИВД+ЭВД -0.0013120885403889984
dQ/Q ПЕВД+ИВД 0.0018553577416087117
dQ/Q ПЕВД+ИВД -0.000600206650047973
dQ/Q ПЕВД+ИВД+ЭВД 0.00026292484854650795
Степень 

In [2]:
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar,Tрасч,Tбал
GTU-PEVD,522.800000,0.1,936.681566,516.2,0.7803,0.1237,0.0301,0.0594,0.0065,542.100000,542.10
PEVD-IVD,456.193690,0.1,860.856095,516.2,0.7803,0.1237,0.0301,0.0594,0.0065,468.083435,468.05
IVD-EVD,308.685074,0.1,697.104142,516.2,0.7803,0.1237,0.0301,0.0594,0.0065,309.200000,309.20
EVD-PPND,235.099443,0.1,617.476312,516.2,0.7803,0.1237,0.0301,0.0594,0.0065,236.591399,235.90
PPND-IND,232.104733,0.1,614.262152,516.2,0.7803,0.1237,0.0301,0.0594,0.0065,233.604351,233.00
IND-GPK,175.324834,0.1,553.674225,516.2,0.7803,0.1237,0.0301,0.0594,0.0065,175.500000,175.50
GPK-out,107.193337,0.1,481.740627,516.2,0.7803,0.1237,0.0301,0.0594,0.0065,112.492608,106.90


In [3]:
water_streams

,T,P,H,G,Tрасч,Gрасч,Tбал,Gбал
PEVD-DROS,496.987439,8.012652,3391.871578,60.700492,511.500000,62.370779,511.500000,63.40
IVD-PEVD,297.938037,8.341200,2753.507839,60.700492,301.297556,62.370779,301.297556,63.40
EVD-IVD,297.938037,8.341200,1374.889555,60.700492,298.297556,62.370779,298.800000,63.40
PEN-EVD,165.671153,8.341200,704.502121,60.700492,164.738361,62.370779,166.500000,63.40
BND-PEN,164.512241,0.692500,695.109778,60.700492,164.738361,62.370779,164.738361,63.40
PPND-TURB,207.793261,0.685915,2863.339801,16.255609,212.291399,14.465753,211.600000,14.76
IND-PPND,164.512241,0.692500,2762.297432,16.255609,164.738361,14.465753,164.738361,14.76
GPK-IND,164.512241,0.692500,729.433743,76.956101,161.738361,76.836531,164.600000,78.16
X-GPK,60.000000,0.692500,251.748396,76.956101,60.000000,76.836531,60.000000,78.16
GPK-RECIRC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


dQ/Q ППНД+ИНД 2.544374969550661e-05
